In [19]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # GPU 메모리를 필요할 때만 할당
    except RuntimeError as e:
        print(e)

# Faster R-CNN 모델 로드
model = hub.load("https://www.kaggle.com/models/rishitdagli/faster-rcnn-cppe5/TensorFlow2/faster-rcnn-cppe5/1")
# 이미지 디렉토리 경로 설정 (테스트 이미지들이 있는 폴더)
image_dir = '/home/elicer/FIshing_vessel/imgdata'  # 이 부분을 실제 이미지 폴더 경로로 수정하세요

# 이미지 파일들 반복 처리
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # 확장자 필터링
        image_path = os.path.join(image_dir, filename)

        # 이미지 불러오기 및 전처리
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환

        # 여기에서 이미지 크기를 조절합니다.
        image_resized = cv2.resize(image_rgb, (120, 120))  # 입력 이미지를 더 작은 크기로 줄임 (예: (200, 200))

        # 모델 입력을 위한 텐서로 변환
        image_tensor = tf.convert_to_tensor(image_resized, dtype=tf.uint8)

        # 배치 차원 추가 (배치 크기 = 1로 설정)
        image_tensor = tf.expand_dims(image_tensor, axis=0)  # 배치 차원 추가 (배치 크기 1)

        # 예측 수행
        results = model(image_tensor)
        result = {key: value.numpy() for key, value in results.items()}

        # 박스, 클래스, 점수 추출
        boxes = result["detection_boxes"][0]  # [ymin, xmin, ymax, xmax]
        classes = result["detection_classes"][0].astype(np.int32)
        scores = result["detection_scores"][0]

        # 예측 결과 시각화
        height, width, _ = image_rgb.shape
        for i in range(len(scores)):
            if scores[i] > 0.5:  # 신뢰도 50% 이상만 표시
                ymin, xmin, ymax, xmax = boxes[i]
                start_point = (int(xmin * width), int(ymin * height))
                end_point = (int(xmax * width), int(ymax * height))
                color = (0, 255, 0)  # 초록색
                thickness = 2
                cv2.rectangle(image_rgb, start_point, end_point, color, thickness)

                # 클래스 정보 표시
                label = f"Class {classes[i]}, Score: {scores[i]:.2f}"
                cv2.putText(image_rgb, label, (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 결과 이미지 시각화
        plt.figure(figsize=(10, 10))
        plt.imshow(image_rgb)
        plt.title(f"Results for {filename}")
        plt.axis('off')
        plt.show()


Physical devices cannot be modified after being initialized


TypeError: Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (<tf.Tensor: shape=(1, 120, 120, 3), dtype=uint8, numpy=
array([[[[ 27,  26,  27],
         [ 33,  36,  46],
         [ 18,  17,  28],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        [[109, 108, 105],
         [ 35,  38,  42],
         [ 18,  18,  22],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        [[ 98,  98,  93],
         [ 38,  39,  37],
         [  9,   9,   8],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        ...,

        [[  1,   1,   1],
         [  1,   1,   1],
         [  6,   6,   6],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  6,   6,   6],
         ...,
         [  1,   1,   1],
         [  1,   1, 